### 웹캠으로 포즈 인식해서 홈트레이닝 자세 코칭 웹앱

In [2]:
from flask import Flask, render_template, Response
import cv2
import mediapipe as mp
from openai import OpenAI
import numpy as np
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=API_KEY)

app = Flask(__name__)

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 비디오 캡처 초기화
cap = cv2.VideoCapture(0)

# 자세 체크 함수
def check_squat(landmarks):
    if landmarks is None:
        return "No person detected."

    # 엉덩이와 무릎 좌표 가져오기
    hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

    # 무릎 각도 계산 (힙-무릎-발목)
    a = np.array([hip.x, hip.y])
    b = np.array([knee.x, knee.y])
    c = np.array([ankle.x, ankle.y])

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    angle = np.degrees(angle)

    if angle < 70:
        return "Squat too shallow!"
    elif 70 <= angle <= 120:
        return "Good squat!"
    else:
        return "Squat too deep!"

# OpenAI로 격려 문구 생성
def generate_encouragement(status):
    prompt = f"운동 자세 상태: '{status}'. 사용자를 짧게 칭찬하거나 조언하는 한 문장을 만들어줘."

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def gen_frames():
    while True:
        success, frame = cap.read()
        if not success:
            break

        # 좌우 반전
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)

        status_text = "No person detected."

        if results.pose_landmarks:
            status_text = check_squat(results.pose_landmarks.landmark)

            # 랜드마크 그리기
            mp.solutions.drawing_utils.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

        # 상태 텍스트 화면에 쓰기
        cv2.putText(frame, status_text, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Apr/2025 15:55:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2025 15:55:43] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2025 15:55:43] "GET /favicon.ico HTTP/1.1" 404 -
